In [16]:
import genanki
import csv
import argparse
import os
import hashlib
import time
from pathlib import Path
import logging

from src.anki import read_file, make_deck, make_model, make_note, make_package, read_package
from src.media import add_images, add_phonetics
from src.log_config import configure_logging

configure_logging(ignore_libs=["PIL", "phonemizer"])

INPUT_DIR = Path("input/")
OUTPUT_DIR = Path("output/")
TEMPLATE_DIR = Path("template/")
MEDIA_DIR = Path("media/")
IMG_DIR = MEDIA_DIR / "img"
AUDIO_DIR = MEDIA_DIR / "sound"
for d in [INPUT_DIR, OUTPUT_DIR, TEMPLATE_DIR, MEDIA_DIR, IMG_DIR, AUDIO_DIR]:
    d.mkdir(parents=True, exist_ok=True)


file_name = "Le_nombres"
file_name = "Cours_01"
file_name = "Cours_01_QA"

file_type = ".md"
df_file = INPUT_DIR / (file_name + file_type)
apkg_file = OUTPUT_DIR / (file_name + ".apkg")
style = TEMPLATE_DIR / "style.css"
img_files, sound_files = [],[]


In [17]:
df = read_file(df_file)
df = add_phonetics(df)
df, img_files = add_images(df, IMG_DIR, force_replace=False)
deck = make_deck(df)
model = make_model(df, style)
successful_cards, total_rows = make_note(deck, model, df)
media_files = img_files + sound_files
make_package(deck, media_files=img_files, apkg_file=apkg_file)
logging.info(f"Created {successful_cards} cards out of {total_rows} possible lines.")
df.head()

[23:57:54] [INFO] [media] - Added phonetics for 33 words.
[23:57:54] [INFO] [anki] - Front-to-back and back-to-front cards will be created.
[23:57:54] [INFO] [root] - Created 33 cards out of 33 possible lines.


,Front,Back,Extra,More,Phonetics,Audio,Reverse
0,zéro,0,,,/zeʁˈo/,,
1,un,1,,,/ˈœ̃/,,
2,deux,2,,,/dˈø/,,
3,trois,3,,,/tʁwˈa/,,
4,quatre,4,,,/kˈatʁ/,,


In [18]:
def _field(field_name):
    # in a f-string, a single '{' is escaped by doubling it
    return (f'{{{{{field_name}}}}}' if field_name in df.columns else '')

def _div(field_name):
    return (f'<div class="{field_name.lower()}">{_field(field_name)}</div>' if field_name in df.columns else '')

_div("Front")

'<div class="front">{{Front}}</div>'